In [2]:
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime

In [12]:
VIIRS_FILE = r"C:\Users\Ankit\Downloads\filtered_viirs.csv"
ERA5_FILE = r"C:\Users\Ankit\Downloads\merged_era5_2015_2016.nc"
OUTPUT_FILE = r"C:\Users\Ankit\Downloads\updated_viirs_binary_fire_2015_2016.nc"

In [13]:
ds = xr.open_dataset(ERA5_FILE)

latitudes = ds.latitude.values
longitudes = ds.longitude.values
times = pd.to_datetime(ds.valid_time.values)

📦 Loading ERA5 grid...


In [14]:
viirs_df = pd.read_csv(VIIRS_FILE)

📄 Reading VIIRS fire detections...


In [15]:
viirs_df.head()

,Unnamed: 0,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,72176,29.09680,79.01381,329.99,0.45,0.39,2015-01-01,744,N,VIIRS,n,2,286.98,2.38,D,0
1,72177,28.84811,77.79187,331.86,0.51,0.41,2015-01-01,744,N,VIIRS,n,2,285.87,3.74,D,0
2,72178,28.98106,77.89695,330.35,0.50,0.41,2015-01-01,744,N,VIIRS,n,2,280.03,3.30,D,0
3,72179,29.07977,77.24300,326.44,0.53,0.42,2015-01-01,744,N,VIIRS,n,2,281.06,3.85,D,0
4,72180,28.97446,78.87379,326.37,0.45,0.39,2015-01-01,744,N,VIIRS,n,2,293.11,3.08,D,0


In [16]:
print(f"🧯 Fire points in 2015–2016: {len(viirs_df)}")

🧯 Fire points in 2015–2016: 68590


In [17]:
binary_raster = np.zeros((len(times), len(latitudes), len(longitudes)), dtype=np.uint8)

🧱 Initializing raster...


In [18]:
def get_nearest_idx(value, array):
  return np.abs(array - value).argmin()

In [19]:
print("🔥 Mapping VIIRS points to raster grid...")

🔥 Mapping VIIRS points to raster grid...


In [20]:

viirs_df["acq_time"] = viirs_df["acq_time"].astype(str).str.zfill(4)
hours = viirs_df["acq_time"].str[:2]
minutes = viirs_df["acq_time"].str[2:]
viirs_df["datetime"] = pd.to_datetime(viirs_df["acq_date"] + " " + hours + ":" + minutes)


dropped = 0
for _, row in viirs_df.iterrows():
    try:
       
        time_idx = get_nearest_idx(row["datetime"], times)
        lat_idx = get_nearest_idx(row["latitude"], latitudes)
        lon_idx = get_nearest_idx(row["longitude"], longitudes)

        binary_raster[time_idx, lat_idx, lon_idx] = 1
    except IndexError:
        dropped += 1
        continue

print(f"Done. Dropped {dropped} out-of-range points.")

print(f"💾 Saving binary raster to {OUTPUT_FILE}...")
fire_da = xr.DataArray(
    binary_raster,
    coords={"time": times, "latitude": latitudes, "longitude": longitudes},
    dims=["time", "latitude", "longitude"],
    name="fire_label"
)

fire_da.to_netcdf(OUTPUT_FILE)
print("Finished: Binary raster saved.")

✅ Done. Dropped 0 out-of-range points.
💾 Saving binary raster to C:\Users\Ankit\Downloads\updated_viirs_binary_fire_2015_2016.nc...
🎉 Finished: Binary raster saved.
